# Análise Completa do Dataset Titanic

Este notebook realiza carregamento, pré-processamento, modelagem, avaliação, clustering e regras de associação sobre o dataset Titanic.

## 1. Importação das bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from yellowbrick.classifier import ConfusionMatrix
import warnings
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules

# Ignorar warnings para evitar poluição visual
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

## 2. Carregamento e pré-processamento dos dados

In [ ]:
# Carregar o arquivo train.csv (deve estar na mesma pasta do notebook)
df_treino = pd.read_csv('train.csv')

# Tratamento de valores ausentes
df_treino['Age'] = df_treino['Age'].fillna(df_treino['Age'].median())
df_treino['Embarked'] = df_treino['Embarked'].fillna(df_treino['Embarked'].mode()[0])
df_treino['Fare'] = df_treino['Fare'].fillna(df_treino['Fare'].median())

# Extração de títulos
df_treino['Title'] = df_treino['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

# Codificação de variáveis categóricas
df_treino = pd.get_dummies(df_treino, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

# Visualizar as primeiras linhas
df_treino.head()

## 3. Seleção das features e divisão dos dados

In [ ]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] + \
           [col for col in df_treino.columns if col.startswith('Sex_') or 
                                               col.startswith('Embarked_') or
                                               col.startswith('Title_')]

X = df_treino[features]
y = df_treino['Survived']

# Divisão em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar dimensões
X_treino.shape, X_teste.shape

## 4. Modelagem: Árvore de Decisão com ajuste de hiperparâmetros

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, None],
    'max_features': [0.8, 'sqrt', None]
}

grid = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)
grid.fit(X_treino, y_treino)

print("Melhores hiperparâmetros:", grid.best_params_)
print("Melhor score de validação:", grid.best_score_)

## 5. Avaliação da Árvore de Decisão

In [ ]:
modelo_final = grid.best_estimator_
y_pred = modelo_final.predict(X_teste)

print(classification_report(y_teste, y_pred))

# Visualização da matriz de confusão
cm = ConfusionMatrix(modelo_final)
cm.fit(X_treino, y_treino)
cm.score(X_teste, y_teste)
cm.show()

## 6. Modelagem: Random Forest e avaliação

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_treino, y_treino)
y_pred_rf = rf.predict(X_teste)

print(classification_report(y_teste, y_pred_rf))

# Visualização da matriz de confusão
cm_rf = ConfusionMatrix(rf)
cm_rf.fit(X_treino, y_treino)
cm_rf.score(X_teste, y_teste)
cm_rf.show()

## 7. Análise de agrupamento: KMeans + PCA

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)

df_treino['Cluster'] = clusters

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=clusters, palette='Set1')
plt.title('Clusters de passageiros (KMeans) com PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

print(df_treino.groupby('Cluster')[['Survived', 'Sex_male', 'Pclass']].mean())

## 8. Regras de Associação com Apriori

In [ ]:
df_assoc = df_treino.copy()
df_assoc['Sex'] = np.where(df_assoc['Sex_male']==1, 'male', 'female')
df_assoc['Class'] = df_assoc['Pclass'].astype(str)
df_assoc['Survived'] = df_assoc['Survived'].astype(str)

df_assoc = df_assoc[['Sex', 'Class', 'Survived']]
df_onehot = pd.get_dummies(df_assoc)

frequent_itemsets = apriori(df_onehot, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(3)